In [1]:
import os
import shutil
import argparse
import os
import sys



from keras_retinanet.preprocessing.csv_generator import CSVGenerator
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
from tensorflow import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time
from keras_retinanet.models import load_model

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# use this to change which GPU to use
# gpu = '0'

# # set the modified tf session as backend in keras
# setup_gpu(gpu)

2023-12-15 21:07:28.247897: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-15 21:07:28.247989: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-15 21:07:28.248085: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-15 21:07:28.275012: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-15 21:07:32.823288: W tensorflow/compiler/

### Post Training Quantization


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

In [3]:
# model_keras = load_model(
#     'path_to_h5_file', backbone_name='resnet50')
model_keras = load_model(
    '/home/developer/Documents/CMU/Capstone_Codes/cervical-cancer-project/PyAVE_cmu_fall2023/cmu_fall2023/assets/resnet50_csv_26.h5', backbone_name='resnet50')
#dynamic quantization not supported so we need to fix the input size
new_input_shape = (1067, 800, 3)
model_keras.inputs[0].set_shape((None, *new_input_shape))
model_keras = models.convert_model(model_keras)
# Rebuild the model to apply the changes
model_keras = keras.models.Model(
    inputs=model_keras.inputs, outputs=model_keras.outputs)

2023-12-15 21:10:39.139334: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-15 21:10:39.493428: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-15 21:10:39.494138: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [ ]:
## Check model summary to understand layers
model_keras.summary()

In [5]:
import io
summary = io.StringIO()
model_keras.summary(print_fn=lambda x: summary.write(x + '\n'))
model_summary = summary.getvalue()
summary.close()
with open('model_summary.txt', 'w') as file:
    file.write(model_summary)

print("Model summary written to 'model_summary.txt'")

Model summary written to 'model_summary.txt'


In [11]:
input_layer = model_keras.get_layer('input_1').output

## Get last feature layer
feature_layer = model_keras.get_layer('C5_reduced').output 

# Getting the regression and classification submodels
regression_submodel = model_keras.get_layer('regression_submodel')
classification_submodel = model_keras.get_layer('classification_submodel')

# Applying P3 layer to both submodels
regression_output = regression_submodel(feature_layer)
classification_output = classification_submodel(feature_layer)

# Now create a new model
new_model = Model(inputs=input_layer, outputs=[regression_output, classification_output])

# Use 'new_model' for further operations

In [ ]:
new_model.summary()

In [ ]:
image = Image.open(
    '../path_to_test_image.jpg').convert('RGB')
image = image.rotate(90)

image = np.asarray(image)
image = image[:, :, ::-1].copy()

image = preprocess_image(image)
image, scale = resize_image(image)
print(image.shape)
# Run the inference
start = time.time()
scores = new_model.predict_on_batch(np.expand_dims(image, axis=0))
inference_time = time.time() - start
print(inference_time)
# predicted_label = labels[0][np.argmax(scores[0])]
# confidence = scores[0][np.argmax(scores[0])]

In [ ]:
def create_generator():
    """ Create generators for evaluation.
    """
    common_args = {

        'preprocess_image': preprocess_image,

    }

    validation_generator = CSVGenerator(
        'calib_images.csv',
        'class.csv',
        shuffle_groups=False,
        **common_args
    )

    return validation_generator

In [ ]:
import pathlib

tflite_models_dir = pathlib.Path("/tmp/mnist_tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
gen = create_generator()
# create an image generator with a batch size of 1
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)
test_generator = gen


def represent_data_gen():
    """ it yields an image one by one """
    for ind in range(len(gen.image_data.keys())):
        # it returns (image and label) tuple
        image = test_generator.load_image(ind)
        image = image[:, :, ::-1].copy()
        image = preprocess_image(image)
        image, scale = resize_image(image)

        print(image.shape)
        yield [np.expand_dims(image, axis=0)]  # return only image

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(new_model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = represent_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant = converter.convert()

In [ ]:
"Store model"
tflite_model_quant_file = tflite_models_dir / \
    "path_to_store_quantized_model"
tflite_model_quant_file.write_bytes(tflite_model_quant)